In [1]:
using Random, Distributions
using Plots
# plotlyjs()
# using PlotlyJS
using DataFrames
using StatsBase
using DelimitedFiles
using LinearAlgebra
# using Distances
# using Symbolics
# using Latexify
using LsqFit
using Clustering

In [69]:
plotlyjs()

WebIO._IJuliaInit()

Plots.PlotlyJSBackend()

# Problem Set 6 (Jonathan Fischer using Julia)

## 1) Non-linear fitting

In [77]:
function read_rxnrates()
    readdlm("rate_vs_conc.dat")
end

read_rxnrates (generic function with 1 method)

In [78]:
data = read_rxnrates()

25×2 Matrix{Float64}:
 0.1  2.0947
 0.3  3.02743
 0.5  3.45027
 0.7  3.53353
 0.9  3.66932
 1.1  3.76629
 1.3  3.64593
 1.5  3.65032
 1.7  3.85984
 1.9  3.78112
 2.1  3.87496
 2.3  3.94252
 2.5  3.79529
 2.7  3.81019
 2.9  3.87909
 3.1  3.88815
 3.3  3.91745
 3.5  3.91305
 3.7  3.81003
 3.9  3.87948
 4.1  3.87203
 4.3  3.80416
 4.5  3.84712
 4.7  4.04025
 4.9  4.02233

In [79]:
#construct Michaelis-Menten model using broadcasting 
@. mm_model(S,beta) = (beta[1]*S)/(beta[2]+S)

mm_model (generic function with 1 method)

In [80]:
xdata = data[:,1] #independent data, concentrations S
ydata = data[:,2] #dependent data, reaction rates
beta0 = [4.0, 0.2] #inital guess for parameters Vmax and Km 

2-element Vector{Float64}:
 4.0
 0.2

In [81]:
fit = curve_fit(mm_model,xdata,ydata,beta0) #fit curve using non-linear lsq 

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([3.9910454568457787, 0.08993779127200312], [0.006538516006389994, 0.04309184485229167, -0.06767623122512356, 0.003113386276485386, -0.040864956069228686, -0.07689804951897461, 0.08687240115708361, 0.11496784014494121, -0.06933249974881583, 0.029547537503409327  …  -0.009624336023883018, -0.032293306949893186, -0.021992381955471263, 0.08630474442975933, 0.021605597516531905, 0.0333511263958548, 0.10512277602854647, 0.0657222947217635, -0.1241436118430479, -0.10321446863976735], [0.5264881692508658 -11.062770626469117; 0.7693534884600133 -7.874396414826567; … ; 0.981223599291017 -0.8175696969153982; 0.981976169840106 -0.7854028838322992], true, Float64[])

In [85]:
println("Optimal Vmax: $(fit.param[1]) \nOptimal Km: $(fit.param[2])") #optimal parameters for curve fitting 

Optimal Vmax: 3.9910454568457787 
Optimal Km: 0.08993779127200312


In [91]:
p1 = plot(xdata,ydata, label = "Data", title="Reaction Rate vs. Initial Concentration",legend=:bottomright)
plot!(xdata, mm_model(xdata,fit.param), label = "Fit: Vmax = $(round(fit.param[1],digits = 2)), Km = $(round(fit.param[2],digits=2))")
xlabel!("Concentration [S]")
ylabel!("Reaction Rate v")
savefig("prob1.png")
display(p1)

In [92]:
sigma = stderror(fit)
# to get margin of error and confidence interval of each parameter at 5% significance level:
margin_of_error = margin_error(fit, 0.05)
confidence_inter = confidence_interval(fit, 0.05)

2-element Vector{Tuple{Float64, Float64}}:
 (3.9508499222409044, 4.0312409914506535)
 (0.07913521961267564, 0.1007403629313306)

### d) Variance of parameter fit

In [93]:
println("Variance of Vmax = $(margin_of_error[1])") 
println("Variance of Km = $(margin_of_error[2])")

Variance of Vmax = 0.040195534604874385
Variance of Km = 0.01080257165932748


## 2) Clustering using k-means

In [140]:
Random.seed!(1234)

TaskLocalRNG()

In [141]:
clustdata = rotl90(readdlm("clust_data.dat"))

2×200 Matrix{Float64}:
  2.88312  2.6798    3.81752   3.49016   …  -3.76524  -0.511051  0.616878
 -1.13397  1.18434  -4.53839  -0.194999      5.89073   6.27067   6.19463

In [142]:
clusters = kmeans(clustdata, 2)

assigns = assignments(clusters) #get assignments of points to clusters
sizes = counts(clusters) #get cluster sizes
centers = clusters.centers #get cluster centers

2×2 Matrix{Float64}:
  3.16367   -0.222084
 -0.608549   5.84392

In [143]:
p2 = scatter(clustdata[2,:],clustdata[1,:], marker_z = clusters.assignments, color = :lightrainbow, legend=false, title="k-means Clustering")
scatter!(centers[:,2],centers[:,1],markercolor= :black, markersize = 10)
xlabel!("x values")
ylabel!("y values")
savefig("prob2.png")
display(p2)

## 3) Clustering and Markov State Models

In [98]:
xtraj = readdlm("Xvstime.dat")
ytraj = readdlm("Yvstime.dat")
ztraj = readdlm("Zvstime.dat")
traj = cat(xtraj,ytraj,ztraj, dims=3) #trajectory matrix 

10×500001×3 Array{Float64, 3}:
[:, :, 1] =
 -34.7416  -34.7436  -34.7456  -34.7475  …  -45.0737  -45.0716  -45.0695
 -35.4515  -35.4576  -35.4636  -35.4697     -46.1405  -46.1428  -46.1453
 -36.0034  -36.0022  -36.001   -35.9997     -47.0515  -47.0512  -47.0508
 -35.8079  -35.8087  -35.8096  -35.8104     -45.9729  -45.9723  -45.9714
 -34.9422  -34.9448  -34.9475  -34.9501     -45.981   -45.9795  -45.9779
 -34.6659  -34.6582  -34.6504  -34.6427  …  -47.0449  -47.0473  -47.0496
 -34.2428  -34.2455  -34.2481  -34.2508     -46.4067  -46.4045  -46.4025
 -35.1962  -35.1934  -35.1908  -35.1881     -47.1033  -47.1035  -47.1037
 -34.4876  -34.4811  -34.4746  -34.4682     -46.1683  -46.1634  -46.1585
 -34.4024  -34.4064  -34.4105  -34.4146     -45.6626  -45.6593  -45.656

[:, :, 2] =
 -106.491  -106.482  -106.473  -106.464  …  -136.075  -136.077  -136.079
 -106.831  -106.833  -106.835  -106.838     -134.976  -134.976  -134.975
 -106.82   -106.826  -106.831  -106.836     -134.27   -134.27   -134.

In [99]:
function get_internaldistances(traj) #measure internal distances and return feature matrix for clustering 
    features = Array{Float64}(undef,7,size(traj)[2])
    for (i,t) in enumerate(eachslice(traj, dims=2)) #get each internal distance and update feature matrix 
        features[1,i] = norm(t[1,:]-t[10,:]) 
        features[2,i] = norm(t[1,:]-t[4,:])
        features[3,i] = norm(t[1,:]-t[5,:])
        features[4,i] = norm(t[2,:]-t[6,:])
        features[5,i] = norm(t[4,:]-t[7,:])
        features[6,i] = norm(t[5,:]-t[10,:])
        features[7,i] = norm(t[5,:]-t[9,:])
    end
    return features 
end

get_internaldistances (generic function with 1 method)

### a) Define each structure by vector of 7 internal distances

In [100]:
features = get_internaldistances(traj) #internal distances x timesteps matrix 

7×500001 Matrix{Float64}:
 2.56024  2.56817  2.57601  2.58393  2.59188  …  3.16938  3.17205   3.17468
 1.99799  2.00604  2.01419  2.02242  2.03073     2.78243  2.78909   2.7958
 1.68688  1.69755  1.70828  1.71904  1.72988     3.46962  3.47411   3.47861
 2.53469  2.53906  2.54328  2.54769  2.55201     2.47079  2.47411   2.47718
 2.05153  2.0499   2.04845  2.04698  2.04539     0.944    0.944072  0.945135
 1.22834  1.22489  1.22144  1.21806  1.21458  …  1.25649  1.25389   1.25127
 1.75366  1.75945  1.76514  1.77088  1.77662     2.38161  2.38074   2.37986

### b) k-means clustering -> 7 x 6 matrix of cluster centers (Julia's kmeans algorith is column-wise rather than row-wise)

In [145]:
Random.seed!(1234)
stateclusters = kmeans(features, 6) #cluster into 6 clusters 
stateclusters.centers

7×6 Matrix{Float64}:
 1.97807  3.10196  3.2726   4.787    1.88152  3.69951
 1.71643  1.71301  2.34365  2.31921  2.03379  1.75439
 1.81977  1.77514  2.87895  2.81168  2.42261  1.90579
 2.00613  1.86085  2.5985   2.6843   1.89473  1.92745
 1.94198  1.78838  1.79827  2.04948  1.68483  1.96883
 1.60974  2.27     1.82777  2.77058  2.65857  3.42
 1.66893  2.00983  1.83306  2.31235  2.49602  2.92559

### c) Calculate transition matrix

In [146]:
function count_trans(clusters; lag=50) #counts frequency of each state, looping through cluster assignments with lag of 50 to preserve markovian behavior 
    trans_matrix = zeros((6,6))
    a_prev = clusters.assignments[1]
    for a in clusters.assignments[1:lag:end] #loop through cluster assignments with step size of 50 
        trans_matrix[a_prev,a] += 1 #update transition matrix at index [from, to]
        a_prev = a 
    end

    for (i,row) in enumerate(eachrow(trans_matrix)) #normalize each row of transition matrix
        trans_matrix[i,:] = row./sum(row)
    end
    return trans_matrix
end

count_trans (generic function with 1 method)

In [147]:
trans_matrix = count_trans(stateclusters) #transition matrix where row is previous state, column is future state 

6×6 Matrix{Float64}:
 0.850959   0.0580098  0.0325747   0.0         0.058456   0.0
 0.0675338  0.789895   0.0530265   0.00850425  0.0310155  0.050025
 0.0421649  0.0585274  0.814349    0.0440529   0.0352423  0.00566394
 0.0        0.0189702  0.0623306   0.869015    0.0        0.0496838
 0.074095   0.0339367  0.0226244   0.0         0.84276    0.0265837
 0.0        0.0894372  0.00616808  0.0447186   0.0223593  0.837317

In [148]:
function findmaxes(trans_matrix)
    maxprobs = []
    for (i,row) in enumerate(eachrow(trans_matrix))
        # newrow = row[1:end .!=i] #new row excludes diagonal element 
        newrow = replace(row, row[i] => 0.0)
        push!(maxprobs,"Most probable transition from $(i): $(findmax(newrow)[2]) with probability $(findmax(newrow)[1])")
    end
    return maxprobs
end

findmaxes (generic function with 1 method)

Most probable transitions, excluding self -> self

In [149]:
findmaxes(trans_matrix)

6-element Vector{Any}:
 "Most probable transition from 1: 5 with probability 0.05845604640785364"
 "Most probable transition from 2: 1 with probability 0.06753376688344172"
 "Most probable transition from 3: 2 with probability 0.05852737570799245"
 "Most probable transition from 4: 3 with probability 0.06233062330623306"
 "Most probable transition from 5: 1 with probability 0.07409502262443439"
 "Most probable transition from 6: 2 with probability 0.08943716268311488"

Why some transitions more probable than others? Structure

In [151]:
clust6idx = findfirst(x->x==6,stateclusters.assignments)
clust2idx = findfirst(x->x==2,stateclusters.assignments)
clust1idx = findfirst(x->x==1,stateclusters.assignments)

c6plot = scatter(traj[:,clust6idx,1],traj[:,clust6idx,2],traj[:,clust6idx,3], mode="markers+lines", title = "Cluster 6", color = :black, legend = false)
c2plot = scatter(traj[:,clust2idx,1],traj[:,clust2idx,2],traj[:,clust2idx,3], mode="markers+lines", title = "Cluster 2", color = :green, legend = false)
c1plot = scatter(traj[:,clust1idx,1],traj[:,clust1idx,2],traj[:,clust1idx,3], mode="markers+lines", title = "Cluster 1", color = :red, legend = false)

l = @layout [a b; a c]
p3 = plot(c6plot, c2plot, c6plot, c1plot, layout = l, size = (700,600), plot_title = "Comparison of Cluster 6 to 2/1", legend = false)
savefig("prob3.png")
display(p3)

In [156]:
println("Comparing the transition of cluster 6 to 2 (p = $(trans_matrix[6,2]) vs 6 to 1 (p = $(trans_matrix[6,1])), as well as between other row extrema from the transition matrix, favored transitions conserve internal distances compared to disfavored transitions")
println("This isn't especially clear from the figure, but the quantitative comparison below supports this point better")

Comparing the transition of cluster 6 to 2 (p = 0.08943716268311488 vs 6 to 1 (p = 0.0), as well as between other row extrema from the transition matrix, favored transitions conserve internal distances compared to disfavored transitions
This isn't especially clear from the figure, but the quantitative comparison below supports this point better


In [157]:
println("Comparing difference in center coordinates between clusters 6 -> 2 vs. 6 -> 1, the difference in transition probability can be better understood")

Comparing difference in center coordinates between clusters 6 -> 2 vs. 6 -> 1, the difference in transition probability can be better understood


In [158]:
stateclusters.centers[:,6] - stateclusters.centers[:,2] #6 -> 2. 
#Only one internal distance larger than 1

7-element Vector{Float64}:
 0.5975504072146842
 0.04137716304367167
 0.13065268084126958
 0.06659791891805411
 0.18045314912402177
 1.1500061666566932
 0.9157653161242876

In [159]:
stateclusters.centers[:,6] - stateclusters.centers[:,1] #6 -> 1 
#Multiple internal distances larger than 1, nearly half in fact 

7-element Vector{Float64}:
  1.721441811315709
  0.03795629130033773
  0.08602071195466454
 -0.07867669327259286
  0.02685003571751854
  1.8102665864224416
  1.2566592253833588

### d) Probability of being in each state

In [160]:
function find_steadystate(trans_matrix) #function iterates state matrix by transition matrix until convergence 
    prevstate = [1,0,0,0,0,0] #initial state 

    while prevstate != vec(prevstate' * trans_matrix) #loop until next state matches previous state 
        prevstate = vec(prevstate' * trans_matrix) 
    end

    return prevstate
end

find_steadystate (generic function with 1 method)

In [164]:
probs = find_steadystate(trans_matrix)
for (i,clust) in enumerate(probs)
    println("Probability of state $(i): $(clust)")
end

Probability of state 1: 0.2234849778779593
Probability of state 2: 0.19987056269593056
Probability of state 3: 0.15936448137935272
Probability of state 4: 0.1108671945627782
Probability of state 5: 0.1766750245126744
Probability of state 6: 0.12973775897130285


### e) Time lag of 100 steps

In [165]:
#Transition matrix using a time lag of 100 steps
count_trans(stateclusters; lag=100)

6×6 Matrix{Float64}:
 0.726465   0.105684   0.0603908  0.0         0.10746    0.0
 0.115423   0.634826   0.0915423  0.0218905   0.0517413  0.0845771
 0.0901015  0.0951777  0.659898   0.0774112   0.0609137  0.0164975
 0.0        0.0448029  0.105735   0.756272    0.0        0.09319
 0.133183   0.0541761  0.0428894  0.00225734  0.716704   0.0507901
 0.0031348  0.15674    0.0188088  0.0799373   0.0470219  0.694357

In [166]:
#Squared transition matrix using time lag of 50 steps 
count_trans(stateclusters; lag=50)^2

6×6 Matrix{Float64}:
 0.733754    0.099076   0.0586456  0.00192834  0.101955    0.00464042
 0.115347    0.636643   0.0888077  0.0186808   0.0575727   0.0829487
 0.0767814   0.0988764  0.671219   0.074908    0.062807    0.0154084
 0.00390929  0.0395615  0.106238   0.760317    0.00389594  0.0860791
 0.128742    0.0634068  0.0418683  0.00247406  0.71702     0.0464885
 0.00795682  0.147501   0.0182233  0.0773371   0.0405566   0.708425

In [167]:
println("Pretty close to perfectly markovian but not exact past two decimal places")

Pretty close to perfectly markovian but not exact past two decimal places


## 4) Local Optimization using Steepest Descent

In [168]:
function loadbeads() #load protein coordinate data into matrix R, with species (H,P) into seq vector 
    oneseq = readdlm("onesequence_-22.79.dat")
    R = oneseq[2:end,2:end]
    seq = oneseq[2:end,1]
    return R, seq 
end

loadbeads (generic function with 1 method)

In [169]:
function calc_LJ(r, epsilon; sigma = 1) # calculate Lennard_Jones potential for a pairwise distance
    4*epsilon*((sigma/r)^12 - (sigma/r)^6) 
end

calc_LJ (generic function with 1 method)

In [170]:
function calc_harmU(r; k=20,l=1) #calculate bonded energy potential
    0.5k*(r-l)^2
end

calc_harmU (generic function with 1 method)

In [171]:
function calc_LJforce(r, epsilon; sigma = 1) #calculate LJ force (gradient)
    48*epsilon*((sigma^12/r^13)-0.5*(sigma^6/r^7))
end

calc_LJforce (generic function with 2 methods)

In [172]:
function calc_harmforce(r;k=20,l=1) #calculate force from covalent bond
    -k*(r-l)
end

calc_harmforce (generic function with 1 method)

In [173]:
function update_F(R, dt, seq) #update gradient matrix F 
    PE = 0 #initialize potential energy
    N = size(R)[1] #number of beads 
    F = zeros(N,3) #initialize force (grad) matrix 

    for i in 1:N-1
        for j in i+1:N
            dr = R[i,:] - R[j,:] #calculate distance vector between two points
            r_mag = norm(dr) #calculate magnitude of distance vector
            if j - i > 1
                if seq[i]*seq[j] == "HH" #if pairwise interaction between H and H, use epsilon = 1
                    epsilon = 1.
                    PE += calc_LJ(r_mag, epsilon)  #accumulate LJ potential energy

                    F[i,:] += calc_LJforce(r_mag,epsilon) * dr / r_mag #update force matrix with gradient for bead i 
                    F[j,:] -= calc_LJforce(r_mag,epsilon) * dr / r_mag #update force matrix with gradient for bead j 
                else #if pairwise interaction not between H and H, use epsilon = 2/3
                    epsilon = 2/3
                    PE += calc_LJ(r_mag, epsilon)  #accumulate LJ potential energy

                    F[i,:] += calc_LJforce(r_mag,epsilon) * dr / r_mag
                    F[j,:] -= calc_LJforce(r_mag,epsilon) * dr / r_mag
                end
            else 
                PE += calc_harmU(r_mag) #accumulate bonded potential energy 
            
                fvector = calc_harmforce(r_mag) * dr / r_mag #get gradient 
                F[i,:] += fvector #update force matrix with gradient for bead i 
                F[j,:] -= fvector #update foce matrix with gradient for bead j 
            end
        end
    end
    return PE, F #return energy and gradient matrix 
end

update_F (generic function with 1 method)

In [174]:
function calc_position(r, a, dt) #update position vector for single particle
    r + (a*dt) #current position plus gradient x timestep 
end

calc_position (generic function with 2 methods)

In [175]:
function update_R!(R, A, dt) #updates position matrix R IN PLACE
    broadcast!(calc_position,R,R,A,dt) #vectorized operation passed R by reference 
end

update_R! (generic function with 2 methods)

In [176]:
function descend(learnrate) #gradient descent function 
    R, seq = loadbeads() #load coordinates, and species sequence
    N = size(R)[1] #number of beads 

    E = 0 #initialize energy 
    dt = 0.001 #trial timestep 
    initE, F = update_F(R,dt,seq) #get initial energy and F 

    converged = false 
    i = 0 #iteration counter 
    maxiter = 100000 #max iterations allowed 

    while ! converged #loop until covergence criterion satisfied 
        update_R!(R,F,dt) #new R updated to t+dt
        newE, F = update_F(R,dt,seq) #new A updated in-place to t+dt, and returns PE into array 
 
        newE > E ? dt = dt*learnrate : dt = dt  #adaptive timestep, if new energy < previous energy, scale timestep by learnrate [0,1]
        E = newE

        if all(x->x<0.01,F) #CONVERGENCE CRITERION: if new gradient is less than some tolerance, accept and end loop
            converged = true 
        end 

        if i == maxiter #time out of while loop if max iterations reached
            return "Max iter reached, final energy was $(E)"
        end
        
        i += 1 #add to iter counter 
    end  

    return initE, E, F, R, i #return initial energy, final energy, force matrix, position matrix, number of iterations 
end

descend (generic function with 2 methods)

In [177]:
initE, E, F, R, i = descend(0.7)

(-22.791333899668558, -66.67340308741058, [0.0023651425500831325 0.0008214279617829168 -0.0013713968125922543; -0.00351496857697442 -0.001965000168323905 0.0017562463447207133; … ; -0.0015765209731071726 -0.004178864718901554 0.0010598532369650304; -0.0026384424038948286 -0.001289625088547408 -0.00047115781754261676], Any[15.217620989872813 -6.792843057297345 7.811714145364959; 15.312931592867873 -7.459630835224506 7.280291327104291; … ; 16.293809738359347 -7.098675318640377 7.530942724938206; 16.236442931964152 -8.052056100344762 7.30583531164843], 8178)

### b) Convergence criteria 

In [178]:
"Convergence criterion to be met was if each element of gradient matrix F was less than tolerance (0.01 in my case), then accept convergence. If all force elements are close to zero, local minimum energy well has been reached"

"Convergence criterion to be met was if each element of gradient matrix F was less than tolerance (0.01 in my case), then accept convergence. If all force elements are close to zero, local minimum energy well has been reached"

### c) Energy minima

In [179]:
"The minimum energy after convergence was $(E), compared to initial energy of $(initE)"

"The minimum energy after convergence was -66.67340308741058, compared to initial energy of -22.791333899668558"

### d) Plot of final configuration

In [180]:
initR, _ = loadbeads()
initconfigplot = scatter(initR[:,1],initR[:,2],initR[:,3], mode="markers+lines",color=:red, title = "Initial protein configuration", legend = false)

finalconfigplot = scatter(R[:,1],R[:,2],R[:,3], mode="markers+lines",color=:green, title = "Final protein configuration", legend = false)

p4 = plot(initconfigplot, finalconfigplot)
savefig("prob4.png")
display(p4)